<a href="https://colab.research.google.com/github/SankalpC10/LLM/blob/main/Music_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **WELCOME TO THIS NOTEBOOK**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Let's install pyspark

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=0d43b232f381644a77eb7f081c550e3bba3707467c1ada97fec1e0afa21cf8a5
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


Importing the modules

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max
from pyspark.ml.feature import  StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder

Creating the spark session


In [ ]:
spark = SparkSession.builder.appName("lastfm").getOrCreate()

# Loading the dataset

In [ ]:
file_path = '/content/drive/MyDrive/dataset/listenings.csv'
df_listenings = spark.read.format('csv').option('header', True).option('inferSchema',True).load(file_path)
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      


# Cleaning tables

In [ ]:
df_listenings = df_listenings.na.drop()
df_listenings.show()

+-----------+-------------+--------------------+---------------+--------------------+
|    user_id|         date|               track|         artist|               album|
+-----------+-------------+--------------------+---------------+--------------------+
|000Silenced|1299680100000|           Price Tag|       Jessie J|         Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|       Jessie J|           Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|          Robyn|            Be Mine!|
|000Silenced|1299679200000|            Acapella|          Kelis|            Acapella|
|000Silenced|1299675660000|   I'm Not Invisible|      The Tease|   I'm Not Invisible|
|000Silenced|1297511400000|Bounce (Feat NORE...|       MSTRKRFT|         Fist of God|
|000Silenced|1294498440000|Don't Stop The Mu...|        Rihanna|Addicted 2 Bassli...|
|000Silenced|1292438340000|               ObZen|      Meshuggah|               ObZen|
|000Silenced|1292437740000|   Yama's Messengers|      

13758905 4



# Let's Perform some aggregation
to see how many times each user has listened to specific track


In [ ]:
row_numbers = df_listenings.count()
column_numbers = len(df_listenings.columns)
print(row_numbers,column_numbers)

13758905 5


In [ ]:
df_listenings_agg = df_listenings.select('user_id','track').groupby('user_id','track').agg(count('*').alias('count')).orderBy('user_id')
df_listenings_agg.show()

+-------+--------------------+-----+
|user_id|               track|count|
+-------+--------------------+-----+
| --Seph| White Winter Hymnal|    3|
| --Seph|         The Funeral|    1|
| --Seph|Hope There's Someone|    1|
| --Seph|         The Painter|    1|
| --Seph|          Je te veux|    1|
| --Seph|            War Pigs|    1|
| --Seph|                 F12|    1|
| --Seph|                Team|    1|
| --Seph|          Nightmares|    1|
| --Seph|               Radio|    1|
| --Seph|   All I Want Is You|    1|
| --Seph|    Little by Little|    2|
| --Seph|        After Nature|    1|
| --Seph|In the Hall of th...|    1|
| --Seph|   Hey There Delilah|    1|
| --Seph|   Let's Call It Off|    1|
| --Seph|               Leloo|    1|
| --Seph|             Pack Up|    1|
| --Seph|           Introitus|    1|
| --Seph|        The Leanover|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [ ]:
row_numbers = df_listenings.count()
column_numbers = len(df_listenings.columns)
print(row_numbers,column_numbers)

13758905 5


In [ ]:
df_listenings_agg = df_listenings_agg.limit(20000)

# Let's convert the user id and track columns into unique integers




In [ ]:
indexer = [StringIndexer(inputCol=col, outputCol=col+ '_index').fit(df_listenings_agg) for col in list(set(df_listenings_agg.columns)-set(['count']))]
pipeline = Pipeline(stages = indexer)
data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)
data.show()

+-------+--------------------+-----+-----------+-------------+
|user_id|               track|count|track_index|user_id_index|
+-------+--------------------+-----+-----------+-------------+
| --Seph|          Nightmares|    1|    10600.0|         69.0|
| --Seph|Virus (Luke Fair ...|    1|    15893.0|         69.0|
| --Seph|Airplanes [feat H...|    1|      521.0|         69.0|
| --Seph|Belina (Original ...|    1|     3280.0|         69.0|
| --Seph|              Monday|    1|      334.0|         69.0|
| --Seph|Hungarian Dance No 5|    1|     7555.0|         69.0|
| --Seph|       Life On Mars?|    1|     1164.0|         69.0|
| --Seph|  California Waiting|    1|      195.0|         69.0|
| --Seph|       Phantom Pt II|    1|     1378.0|         69.0|
| --Seph|   Summa for Strings|    1|    13737.0|         69.0|
| --Seph|      Hour for magic|    2|     7492.0|         69.0|
| --Seph|Hungarian Rhapsod...|    1|     7556.0|         69.0|
| --Seph|     The Way We Were|    1|    14958.0|       

In [ ]:
data = data.select('user_id_index','track_index','count').orderBy('user_id_index')

In [ ]:
# List of columns to index
columns_to_index = list(set(df_listenings_agg.columns) - set(['count']))

# Create StringIndexer instances for each column
indexer = [StringIndexer(inputCol=col, outputCol=col + '_index', handleInvalid="keep").fit(df_listenings_agg) for col in columns_to_index]

# Create a pipeline with the indexers
pipeline = Pipeline(stages=indexer)

# Apply the pipeline to your DataFrame
data = pipeline.fit(df_listenings_agg).transform(df_listenings_agg)

# Show the resulting DataFrame
data.show()

+-------+--------------------+-----+-----------+-------------+
|user_id|               track|count|track_index|user_id_index|
+-------+--------------------+-----+-----------+-------------+
| --Seph|          Nightmares|    1|    10600.0|         69.0|
| --Seph|Virus (Luke Fair ...|    1|    15893.0|         69.0|
| --Seph|Airplanes [feat H...|    1|      521.0|         69.0|
| --Seph|Belina (Original ...|    1|     3280.0|         69.0|
| --Seph|              Monday|    1|      334.0|         69.0|
| --Seph|Hungarian Dance No 5|    1|     7555.0|         69.0|
| --Seph|       Life On Mars?|    1|     1164.0|         69.0|
| --Seph|  California Waiting|    1|      195.0|         69.0|
| --Seph|       Phantom Pt II|    1|     1378.0|         69.0|
| --Seph|   Summa for Strings|    1|    13737.0|         69.0|
| --Seph|      Hour for magic|    2|     7492.0|         69.0|
| --Seph|Hungarian Rhapsod...|    1|     7556.0|         69.0|
| --Seph|     The Way We Were|    1|    14958.0|       

# Train and Test data

In [ ]:
(training,test) = data.randomSplit([0.5,0.5])

# Let's Create our Model

In [ ]:

USER_ID = 'user_id_index'
TRACK = 'track_index'
COUNT = 'count'

als = ALS(maxIter = 5, regParam=0.01, userCol = USER_ID, itemCol = TRACK, ratingCol = COUNT)
model = als.fit(training)

predictions = model.transform(test)


# Generate top 10 Track recommendations for each user

In [ ]:
recs = model.recommendForAllUsers(10)

In [ ]:
recs.show()

+-------------+--------------------+
|user_id_index|     recommendations|
+-------------+--------------------+
|            0|[{15427, 11.90500...|
|            1|[{349, 8.990572},...|
|            2|[{349, 5.7906175}...|
|            3|[{7847, 7.958605}...|
|            4|[{4460, 18.147211...|
|            5|[{13561, 7.203049...|
|            6|[{349, 10.460324}...|
|            7|[{7847, 7.313006}...|
|            8|[{4460, 12.779207...|
|            9|[{802, 4.940565},...|
|           10|[{7847, 7.8536835...|
|           11|[{12695, 8.651566...|
|           12|[{3140, 6.5351753...|
|           13|[{9321, 9.0086565...|
|           14|[{4460, 32.658653...|
|           15|[{4460, 17.079054...|
|           16|[{14299, 8.931053...|
|           17|[{1739, 7.9628067...|
|           18|[{12695, 9.423298...|
|           19|[{15427, 8.205583...|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
recs.take(1)

[Row(user_id_index=0, recommendations=[Row(track_index=15427, rating=11.905009269714355), Row(track_index=2380, rating=7.040561199188232), Row(track_index=6688, rating=6.728918075561523), Row(track_index=1439, rating=5.7273664474487305), Row(track_index=7847, rating=5.55950927734375), Row(track_index=7, rating=5.432389259338379), Row(track_index=2846, rating=5.176091194152832), Row(track_index=165, rating=5.165829658508301), Row(track_index=12191, rating=5.126568794250488), Row(track_index=10338, rating=4.941786289215088)])]